In [ ]:
from flask import Flask,render_template,request
import requests
import pickle
from tensorflow.keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from transformers import pipeline

In [ ]:
def remove_non_ar(text):
    return re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', text)

In [ ]:
stop_words =['من',
 'في',
 'على',
 'و',
 'فى',
 'يا',
 'عن',
 'مع',
 'ان',
 'هو',
 'علي',
 'ما',
 'اللي',
 'كل',
 'بعد',
 'ده',
 'اليوم',
 'أن',
 'يوم',
 'انا',
 'إلى',
 'كان',
 'ايه',
 'اللى',
 'الى',
 'دي',
 'بين',
 'انت',
 'أنا',
 'حتى',
 'لما',
 'فيه',
 'هذا',
 'واحد',
 'احنا',
 'اي',
 'كده',
 'إن',
 'او',
 'أو',
 'عليه',
 'ف',
 'دى',
 'مين',
 'الي',
 'كانت',
 'أمام',
 'زي',
 'يكون',
 'خلال',
 'ع',
 'كنت',
 'هي',
 'فيها',
 'عند',
 'التي',
 'الذي',
 'قال',
 'هذه',
 'قد',
 'انه',
 'ريتويت',
 'بعض',
 'أول',
 'ايه',
 'الان',
 'أي',
 'منذ',
 'عليها',
 'له',
 'ال',
 'تم',
 'ب',
 'دة',
 'عليك',
 'اى',
 'كلها',
 'اللتى',
 'هى',
 'دا',
 'انك',
 'وهو',
 'ومن',
 'منك',
 'نحن',
 'زى',
 'أنت',
 'انهم',
 'معانا',
 'حتي',
 'وانا',
 'عنه',
 'إلي',
 'ونحن',
 'وانت',
 'منكم',
 'وان',
 'معاهم',
 'معايا',
 'وأنا',
 'عنها',
 'إنه',
 'اني',
 'معك',
 'اننا',
 'فيهم',
 'د',
 'انتا',
 'عنك',
 'وهى',
 'معا',
 'آن',
 'انتي',
 'وأنت',
 'وإن',
 'ومع',
 'وعن',
 'معاكم',
 'معاكو',
 'معاها',
 'وعليه',
 'وانتم',
 'وانتي',
 '¿',
 '|']

In [ ]:
def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
      # if read it from file
#     ar_stopwords_list = open("arabic_stopwords.txt", "r") 
#     stop_words = ar_stopwords_list.read().split("\n")
#     stop_words = []
    original_words = []
    words = word_tokenize(text) # it works on one hadith not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence


In [ ]:
def remove_duplicate_letters(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def preprocesss(text):
    text=remove_non_ar(text)
    text=removing_ar_stopwords(text)
    return remove_duplicate_letters(text)
    

In [ ]:
svm_model = pickle.load(open('xgbmodel.pkl', 'rb'))
lstm_model = load_model('lstm_model.h5')
tok = pickle.load(open('tokenizer.pkl', 'rb'))
bert_model = pipeline("text-classification", model="output_dir")

In [ ]:
mapping={'AE': 16,'BH': 17,'DZ': 10,'EG': 11,'IQ': 0,'JO': 6,'KW': 13,'LB': 12,'LY': 1,'MA': 7,'OM': 14,'PL': 3,
         'QA': 2,'SA': 8,'SD': 15,'SY': 4,'TN': 5,'YE': 9}

In [ ]:
app = Flask(__name__)
@app.route('/prediction', methods=['GET','POST'])
def predict():
    text=request.form['text']
    text=preprocesss(text)
    xgb_pred = xgb_model.predict([text])
    encoded = tok.texts_to_sequences([text])
    padded = pad_sequences(encoded,maxlen=280, padding='post')
    lstm_pred = np.argmax(lstm_model.predict(padded))
    lstm_pred = [k for k, v in mapping.items() if v == lstm_pred]
    bert_pred = bert_model(text)
    return render_template('prediction.html', prediction_string = 'Predictions :',
                           svm = 'XGB prediction : {}'.format(svm_pred[0]),
                           lstm = 'LSTM prediction : {}'.format(lstm_pred[0]),
                           arabert = 'Arabert prediction : {}'.format(bert_pred[0]['label']))
    
app.run()